# 各直交位相振幅確認

In [1]:
import numpy as np
from convinience import Spectrum, Band, synthesisSpectrum
from convinience import gaussian_profile, compensation, calc_totalPower ,attenuation
from mso58trans import MSO58Wrapper
from shutter import servo_shutter
from WSMethods import uploadProfile, getWSrange
from sklearn.decomposition import PCA
from csv import DictWriter
import datetime
import copy
from time import sleep
from matplotlib import pyplot as plt

In [ ]:
# 各種設定周り ここにすべてまとめる

class Setting:
    def __init__(self):
        self.ws_ip                          = '169.254.6.8'
        self.ws_Freq_resolution             = 0.001 # 0.001THz = 1GHz
        self.ws_Freqrange                   = None
        self.optSpectrum : Spectrum         = None
        self.mso58_address                  = 'USB::0x0699::0x0530::C043144::INSTR'
        self.homodyne_port                  = 1
        self.fastFramePulseNum              = 5
        self.quadrature_method              = lambda volts: PCA().fit_transform(volts.reshape((-1,volts.shape[-1])))[:,0].reshape(volts.shape[:-1])
        # self.quadrature_method              = lambda volts: volts[:,100]
        self.targetLOPower                  = 1
        self.shutter_port                   = 'COM3'
        self.pumpShutterPort : int          = 2
        self.param_csv                      = f"{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}record.csv"


    def condition_approval(self):
        assert self.ws_Freqrange             is not None
        assert self.optSpectrum              is not None
        assert self.optSpectrum.powerdensity is not None
        assert self.optSpectrum.frequency    is not None
        assert self.optSpectrum.wavelength   is not None
        assert self.optSpectrum.phase        is not None
        assert self.fastFramePulseNum        is not None
        assert self.quadrature_method        is not None
        assert self.targetLOPower            is not None
        assert self.shutter_port             is not None
        assert self.pumpShutterPort          is not None
        assert self.param_csv                is not None


setting = Setting()

startFreq, endFreq    = getWSrange(setting.ws_ip)
setting.ws_Freqrange  = np.arange(startFreq, endFreq, setting.ws_Freq_resolution)

setting.optSpectrum                = Spectrum()
setting.optSpectrum.band           = Band()
setting.optSpectrum.frequency      = np.load('optFreq.npy')
setting.optSpectrum.powerdensity   = np.load('optSpectrum.npy')
setting.optSpectrum.phase          = np.zeros_like(setting.optSpectrum.frequency)

setting.condition_approval()

### 確認

In [ ]:
osc_handler = MSO58Wrapper(setting.mso58_address)
osc_handler.push_single()
t, w = None, None
while True:
    try:
        t, w = osc_handler.transfer2byte(setting.homodyne_port)
        break
    except:
        sleep(1)
        continue
## quad

pulseNum        = setting.fastFramePulseNum
quadrature_func = setting.quadrature_method

frameLength = w.shape[1]
frames      = w.shape[0]
period      = [int(i * frameLength/pulseNum) for i in range(pulseNum)]
duration    = int(frameLength/pulseNum)
quadratures = np.zeros((frames, pulseNum))

volts = np.zeros((frames,pulseNum,duration))

for i in range(pulseNum):
    s = period[i]
    volts[:,i,:] = w[:,s:(s+duration)]

quadratures = quadrature_func(volts)


In [ ]:
plt.plot(volts[:,0,:].T);

In [ ]:
plt.scatter(range(quadratures.shape[0]), quadratures[:,0], s=0.1)